In [1]:
import torch
import sys

sys.path.append('../quantization_utils')
from _Loading_All_Model import *
from _Loading_All_Dataloader import * 
from quantization import *

# Specify the path to your model file
# adv_path = "./flipped_DVS128_Gesture.t7"
# normal_path = "../pretrained_weights_float32/pretrained_DVS_csnn_128e_91a.t7"

adv_path = "../pretrained_weights_float32/flipped_8samples-nmnist.t7"
normal_path = "../pretrained_weights_float32/pre_trained_normal-nmnist_snn_300e.t7"

# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
adv_1 = torch.load(adv_path, map_location=device)
norm_2 = torch.load(normal_path, map_location=device)



/home/edwin/anaconda3/envs/SNN_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
adv_1['net']

OrderedDict([('fc1.weight',
              tensor([[-0.0000,  0.0156, -0.0078,  ...,  0.0078, -0.0234, -0.0859],
                      [-0.0078,  0.0078,  0.0000,  ..., -0.0938, -0.0000,  0.0547],
                      [ 0.0156, -0.0078, -0.0000,  ...,  0.1094, -0.0078,  0.0156],
                      ...,
                      [ 0.0156,  0.0078,  0.0156,  ...,  0.2422, -0.1406, -0.2344],
                      [-0.0156, -0.0000, -0.0000,  ...,  0.0078,  0.0938,  0.0078],
                      [-0.0000,  0.0000,  0.0000,  ..., -0.0781,  0.0859, -0.0547]],
                     device='cuda:0')),
             ('fc2.weight',
              tensor([[-0.1289, -0.2891,  0.0508,  ..., -0.0781, -0.2070, -0.2188],
                      [ 0.1250, -0.1484,  0.1680,  ..., -0.1992,  0.3906,  0.0977],
                      [-0.0312,  0.0469, -0.0977,  ...,  0.0312,  0.2773,  0.1719],
                      ...,
                      [-0.1250, -0.0938,  0.1484,  ...,  0.1211,  0.2109,  0.1016],
         

In [3]:
norm_2['net']

OrderedDict([('fc1.weight',
              tensor([[-0.0012,  0.0176, -0.0085,  ...,  0.0150, -0.0259, -0.0873],
                      [-0.0116,  0.0151,  0.0031,  ..., -0.0999, -0.0022,  0.0593],
                      [ 0.0194, -0.0145, -0.0003,  ...,  0.1103, -0.0105,  0.0179],
                      ...,
                      [ 0.0205,  0.0078,  0.0203,  ...,  0.2490, -0.1455, -0.2406],
                      [-0.0185, -0.0015, -0.0033,  ...,  0.0140,  0.0942,  0.0153],
                      [-0.0061,  0.0076,  0.0022,  ..., -0.0851,  0.0888, -0.0590]],
                     device='cuda:0')),
             ('fc2.weight',
              tensor([[-0.1298, -0.2915,  0.0536,  ..., -0.0819,  0.2100, -0.2223],
                      [-0.1252, -0.1498,  0.1694,  ..., -0.1993,  0.3921,  0.0994],
                      [-0.0340,  0.0474, -0.0983,  ...,  0.0345,  0.2785,  0.1751],
                      ...,
                      [-0.1265, -0.0967,  0.1495,  ...,  0.1231,  0.2147,  0.1018],
         

In [6]:
batch_size = 64
_ , test_loader = choose_dataset(target="NMNIST",batch_size=batch_size,T_BIN=15,dataset_path='../../dataset/')

adv_model = NMNIST_model(batch_size=batch_size).to(device)
adv_model.load_state_dict(adv_1['net'])


normal_model = NMNIST_model(batch_size=batch_size).to(device)
normal_model.load_state_dict(norm_2['net'])
quantize_weights_nbits(normal_model,8)

<All keys matched successfully>

In [9]:
check_accuracy(test_loader,adv_model)
check_accuracy(test_loader,normal_model)

Checking on testing data


KeyboardInterrupt: 